In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('../polygraphpy/data/original_dataset.csv')
inchi_list = df['inchi'].drop_duplicates().values

scaler = MinMaxScaler()
xlogp_list = scaler.fit_transform(df['xlogp'].values.reshape(-1,1)).flatten()